In [ ]:
import sys  
sys.path.insert(0, '../py')
from graviti import *

import json
import numpy as np
from skimage.draw import polygon
from skimage import io
from matplotlib import pyplot as plt
import glob
import pandas as pd
import os
from scipy.sparse import coo_matrix
from skimage.measure import label, regionprops#, regionprops_table
import math

In [ ]:
sample = '/home/garner1/Downloads/TCGA-O1-A52J-01Z-00-DX1.26F6ECCA-D614-4950-98E6-4D76E82F71B4.svs/luad_polygon/TCGA-O1-A52J-01Z-00-DX1.26F6ECCA-D614-4950-98E6-4D76E82F71B4.svs'
patchlist = glob.glob(sample+'/*.csv')

In [ ]:
import timeit
import multiprocessing
from joblib import Parallel, delayed
from datetime import datetime
from tqdm import tqdm

features = ['centroid_x','centroid_y','area','eccentricity','orientation','perimeter','solidity']

num_cores = multiprocessing.cpu_count() # numb of cores
output = Parallel(n_jobs=num_cores)(
    delayed(show_patches_parallel)(filename) for filename in tqdm(patchlist) if ~pd.read_csv(filename).empty
    )

In [ ]:
output = Parallel(n_jobs=num_cores)(
    delayed(measure_patch_of_polygons)(filename,features) for filename in tqdm(patchlist[:]) if ~pd.read_csv(filename).empty
    )

In [ ]:
print('The number of cells is: '+str(cells))
print(data.shape)

In [ ]:
mtx.nonzero()